# Normalisation des données

Afin de permettre le traitement des données sur le cloud Amazon, il nous faut normaliser nos données.

In [ ]:
# Installation des dépendances
%pip install boto3
%pip install smart_open
%pip install python-dotenv
# https://github.com/GeospatialPython/pyshp
%pip install pyshp

## Connexion au bucket S3 Amazon

Pour pouvoir travailler avec nos données, nous utilisons un bucket S3 contenant les dataset. Afin de permettre la connexion au service S3, il est nécessaire d'avoir défini un fichier `.env` contenant les variables d'environnements suivantes :

```
AWS_ACCESS_KEY_ID=xxxxxx
AWS_SECRET_ACCESS_KEY=xxxxxx
```

Ensuite, il suffit d'ouvrir une session à l'aide de la librairie `boto3` :

In [2]:
import os, boto3, dotenv
from smart_open import open

dotenv.load_dotenv('./.env', override=True)

session = boto3.Session(
    aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY'],
)

url = 's3://poc-automatic-land-parcel-recognition/test.txt'

with open(url, 'rb', transport_params = {'client': session.client('s3')}) as reader:
    for line in reader:
        print(line.decode('utf8'))


Content of test.txt file: HELLO WORLD !


## Fichiers Shapefile

L'étape suivante est d'apprendre à traiter les données parcellaires à notre disposition contenues dans un Shapefile.

Le shapefile, ou « fichier de formes » est un format de fichier pour les systèmes d'informations géographiques (SIG). Initialement développé par ESRI pour ses logiciels commerciaux, ce format est désormais devenu un standard de facto, dont les spécifications sont ouvertes. Nous allons donc procéder à la lecture d'un exemple de fichier SHP contenant le "dallage" global de notre zone à couvrir.

Pour cela, nous utilisons la librairie python `pyshp``:

In [3]:
import shapefile

url = 's3://poc-automatic-land-parcel-recognition/dataset-reference-fr/092/'

myshp = open(url + 'dalles.shp', 'rb', transport_params = {'client': session.client('s3')}) # Shapes
mydbf = open(url + 'dalles.dbf', 'rb', transport_params = {'client': session.client('s3')}) # Records
myprj = open(url + 'dalles.prj', 'rb', transport_params = {'client': session.client('s3')}) # Projection
sr = shapefile.Reader(shp=myshp, dbf=mydbf, prj=myprj)

print(sr.bbox) # Bounding Box

for n in range(0, sr.numRecords):
    print(sr.record(n))
    print(sr.shape(n).points)
    
myshp.close()
mydbf.close()
myprj.close()


[635000.0, 6845000.0, 655000.0, 6875000.0]
Record #0: ['./92-2021-0645-6855-LA93-0M20-E080.jp2']
[(645000.0, 6855000.0), (650000.0, 6855000.0), (650000.0, 6850000.0), (645000.0, 6850000.0), (645000.0, 6855000.0)]
Record #1: ['./92-2021-0640-6870-LA93-0M20-E080.jp2']
[(640000.0, 6870000.0), (645000.0, 6870000.0), (645000.0, 6865000.0), (640000.0, 6865000.0), (640000.0, 6870000.0)]
Record #2: ['./92-2021-0650-6860-LA93-0M20-E080.jp2']
[(650000.0, 6860000.0), (655000.0, 6860000.0), (655000.0, 6855000.0), (650000.0, 6855000.0), (650000.0, 6860000.0)]
Record #3: ['./92-2021-0645-6850-LA93-0M20-E080.jp2']
[(645000.0, 6850000.0), (650000.0, 6850000.0), (650000.0, 6845000.0), (645000.0, 6845000.0), (645000.0, 6850000.0)]
Record #4: ['./92-2021-0645-6860-LA93-0M20-E080.jp2']
[(645000.0, 6860000.0), (650000.0, 6860000.0), (650000.0, 6855000.0), (645000.0, 6855000.0), (645000.0, 6860000.0)]
Record #5: ['./92-2021-0650-6870-LA93-0M20-E080.jp2']
[(650000.0, 6870000.0), (655000.0, 6870000.0), (65500

## Normalisation des données

Notre but maintenant est de normaliser notre dataset. En entrée, nous avons les Shapefiles contenant les géométries des parcelles et des fichiers images JP2000 ou GeoTIFF :

![SampleDataOrthophoto.png](doc/SampleDataOrthophoto.png)

En sortie, on doit produire un fichier JSON dit *manifest* tel que défini dans la [documentation Amazon](https://docs.aws.amazon.com/lookout-for-vision/latest/developer-guide/manifest-files.htmll) afin d'avoir une données compréhensible par la suite. Ce fichier JSON définit des zones dans l'image, définies par des coordonnées en pixel, qui correspondent aux éléments reconnus dans l'image. Dans notre cas, il s'agira de parcelles.

Nous devons donc mettre en place un processus qui :
- Va lire les fichiers Shapefile
- Va lire les coordonnées des géométries des parcelles (en prenant en compte la projection)
- Va trouver l'image correspondante
- Va repositionner les points en coordonnées locales en pixel dans l'image
- Va générer le JSON attendu

### Lecture du fichier Shapefile, 

In [6]:
url = 's3://poc-automatic-land-parcel-recognition/dataset-reference-fr/092/'

# Lecture du fichier contenant les dalles
with open(url + 'dalles.shp', 'rb', transport_params = {'client': session.client('s3')}) as dalleshp, open(url + 'dalles.dbf', 'rb', transport_params = {'client': session.client('s3')}) as dalledbf, open(url + 'dalles.prj', 'rb', transport_params = {'client': session.client('s3')}) as dalleprj:
    dallerdr = shapefile.Reader(shp=dalleshp, dbf=dalledbf, prj=dalleprj)

    # Lecture du fichier contenant les parcelles
    with open(url + 'parcelles.shp', 'rb', transport_params = {'client': session.client('s3')}) as parcelleshp, open(url + 'parcelles.dbf', 'rb', transport_params = {'client': session.client('s3')}) as parcelledbf, open(url + 'parcelles.prj', 'rb', transport_params = {'client': session.client('s3')}) as parcelleprj:
        parcelrdr = shapefile.Reader(shp=parcelleshp, dbf=parcelledbf, prj=parcelleprj)

        for n in range(0, 5):
            # On recupère une parcelle
            parcel = parcelrdr.shape(n)

            # On cherche à trouver la dalle correspondante
            for dalle in dallerdr.shapes():
                print(parcel.points)
                print("in")
                print(dalle.bbox)
                print("?")


[(649636.4471738542, 6852112.884597713), (649622.3973377418, 6852120.100487968), (649608.0473582301, 6852127.452447567), (649593.7924777429, 6852134.748022184), (649565.407477983, 6852149.304837236), (649551.3065634868, 6852156.543585095), (649549.5486630904, 6852153.02301074), (649528.9812633416, 6852157.9163176725), (649533.4932261747, 6852176.390289745), (649537.2275636115, 6852191.657624856), (649547.4819008075, 6852189.222350027), (649552.0621511445, 6852207.951484832), (649549.4386241162, 6852212.232886771), (649537.2478123646, 6852215.118639882), (649521.6901405054, 6852218.789798327), (649506.1693325145, 6852222.471797052), (649494.1179142671, 6852225.323071368), (649489.6985940103, 6852222.570651972), (649485.8505136339, 6852206.070118463), (649492.6318188485, 6852199.828905411), (649489.0612808445, 6852183.970861939), (649481.2276803816, 6852185.851452744), (649473.0125833895, 6852187.813208977), (649476.8831407654, 6852203.512964807), (649481.8586639473, 6852224.562488524), 